# The groupby operation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
plt.rc('figure', figsize=(12, 6))
pd.options.display.max_rows = 10

I've provided the reviews by the top 100 reviewers.
We'll use it for talking about groupby.

In [ ]:
df = pd.read_csv("data/subset.csv.gz", parse_dates=['time'])
df.head()

In [ ]:
df.info()

## Aside: Namespaces

Pandas has been expanding its use of namespaces (or accessors) on `DataFrame` to group together related methods. This also limits the number of methods direclty attached to `DataFrame` itself, which can be overwhelming.

Currently, we have these namespaces:

- `.str`: defined on `Series` and `Index`es containing strings (object dtype)
- `.dt`: defined on `Series` with `datetime` or `timedelta` dtype
- `.cat`: defined on `Series` and `Indexes` with `category` dtype
- `.plot`: defined on `Series` and `DataFrames`

# Groupby

Groupby operations come up in a lot of contexts.
At its root, groupby about doing an operation on many subsets of the data, each of which shares something in common.
The components of a groupby operation are:

## Components of a groupby

1. **split** a table into groups
2. **apply** a function to each group
3. **combine** the results into a single DataFrame or Series

In pandas the `split` step looks like

```python
df.groupby( grouper )
```

`grouper` can be many things

- Series (or string indicating a column in `df`)
- function (to be applied on the index)
- dict : groups by *values*
- `levels=[ names of levels in a MultiIndex ]`

## Split

Break a table into smaller logical tables according to some rule

In [ ]:
gr = df.groupby('beer_name')
gr

We haven't really done any actual work yet, but pandas knows what it needs to know to break the larger `df` into many smaller pieces, one for each distinct `beer_name`.

## Apply & Combine

To finish the groupby, we apply a method to the groupby object.

In [ ]:
review_cols = ['review_appearance', 'review_aroma', 'review_overall',
               'review_palate', 'review_taste']

df.groupby('beer_name')[review_cols].agg('mean')

In this case, the function we applied was `'mean'`.
Pandas has implemented cythonized versions of certain common methods like mean, sum, etc.
You can also pass in regular functions like `np.mean`.

In terms of split, apply, combine, split was `df.groupby('beer_name')`. 
We apply the `mean` function by passing in `'mean'`.
Finally, by using the `.agg` method (for aggregate) we tell pandas to combine the results with one output row per group.

You can also pass in regular functions like `np.mean`.

In [ ]:
df.groupby('beer_name')[review_cols].agg(np.mean).head()

Finally, [certain methods](http://pandas.pydata.org/pandas-docs/stable/api.html#id35) have been attached to `Groupby` objects.

In [ ]:
df.groupby('beer_name')[review_cols].mean()

<div class="alert alert-success" data-title="Highest Variance">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Highest Variance</h1>
</div>

<p>Find the `beer_style`s with the greatest variance in `abv`.</p>

- hint: `.std` calculates the standard deviation (`.var` for variance), and is available on `GroupBy` objects like `gr.abv`.
- hint: use `.sort_values` to sort a Series by the values (it took us a while to come up with that name)

In [ ]:
%load solutions/groupby_abv.py

## `.agg` output shape

The output shape is determined by the grouper, data, and aggregation

- Grouper: Controls the output index
    * single grouper -> Index
    * array-like grouper -> MultiIndex
- Subject (Groupee): Controls the output data values
    * single column -> Series (or DataFrame if multiple aggregations)
    * multiple columns -> DataFrame
- Aggregation: Controls the output columns
    * single aggfunc -> Index in the colums
    * multiple aggfuncs -> MultiIndex in the columns (Or 1-D Index if groupee is 1-D)


We'll go into MultiIndexes in a bit, but for know, think of them as regular Indexes with multiple levels (columns).

In [ ]:
# single grouper, single groupee, single aggregation
df.groupby('beer_style').review_overall.agg('mean')

In [ ]:
# multiple groupers, multiple groupee, single aggregation
df.groupby(['brewer_id', 'beer_name'])[review_cols].agg(['mean', 'min', 'max', 'std', 'count'])

<div class="alert alert-success" data-title="Rating by length">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Rating by length</h1>
</div>

<p>Plot the relationship between review length (number of characters) and average `review_overall`.</p>

- Hint: use `.plot(style='k.')`
- We've grouped by columns so far, you can also group by any series with the same length

In [ ]:
%load solutions/groupby_00.py

<div class="alert alert-success" data-title="Reviews by Length">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Reviews by Length</h1>
</div>

<p>Find the relationship between review length (number of **words** and average `review_overall`.)</p>

- Hint: You can pass a [regular expression](https://docs.python.org/3/howto/regex.html#matching-characters) to any of the `.str` methods.

In [ ]:
%load solutions/groupby_00b.py

<div class="alert alert-success" data-title="Rating by number of Reviews">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Rating by number of Reviews</h1>
</div>

<p>Find the relationship between the number of reviews for a beer and the average `review_overall`.</p>


In [ ]:
%load solutions/groupby_01.py